In [1]:
ls ''

 Volume in drive C has no label.
 Volume Serial Number is E842-2BE1

 Directory of c:\Users\sunabove\Documents\KakaoTalk Downloads



File Not Found


In [2]:
# 1. Library install
!pip install librosa
!apt-get install graphviz
!pip install graphviz

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for audioread: filename=audioread-2.1.9-py3-none-any.whl size=23154 sha256=d3ee32fc8b02fc94081405fc2905ce411e799be48602b1553a4644cda4a3dc0f
  Stored in directory: c:\users\sunabove\appdata\local\pip\cache\wheels\a2\a3\bd\ec1568ce7515115a11ab686d509ad302124c782af065de47ee
  Created wheel for resampy: filename=resampy-0.2.2-py3-none-any.whl size=320731 sha256=67a6b98a3c144eb36bec17cabce3eeb137e36b1911c6914a2ca0bfb86641aa90
  Stored in directory: c:\users\sunabove\appdata\local\pip\cache\wheels\86\2c\7d\46a32a246b0e5939cea2c5ec1492164073e0c5d16d666ae2cd
Successfully built audioread resampy


'apt-get' is not recognized as an internal or external command,
operable program or batch file.


Defaulting to user installation because normal site-packages is not writeable


In [ ]:
# 2. import library
"""'''
os,glob : dir와 파일들에 접근하기 위한 Lib
tensorflow, keras : 학습 및 모델 생성을 위한 Lib
numpy : Matrix transpose 등 연산을 휘한 Lib
librosa : feature extraction 및 음성 파일을 처리하기 위한 lib
tqdm : iteration progress bar 생성 lib
sklearn :scikit learn, machine learning을 지원해주는 lib
'''"""

import os
import tensorflow as tf
import keras
import numpy as np
import librosa
import librosa.display
import glob
from tqdm import tqdm_notebook as tqdm
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.utils import np_utils
from keras.utils.vis_utils import model_to_dot
%matplotlib inline 
import matplotlib.pyplot as plt
from sklearn.utils import shuffle


In [ ]:
# 3. 구글 마운트
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
tf.__version__

'2.7.0'

In [ ]:
# colab 사양 확인하기
#1. 플랫폼
import platform
platform.platform()

'Linux-5.4.104+-x86_64-with-Ubuntu-18.04-bionic'

In [ ]:
#2. 운영체제
!cat /etc/issue.net


Ubuntu 18.04.5 LTS


In [ ]:
#3. CPU 사양
!cat /proc/cpuinfo


In [ ]:
#4. 메모리 사양
!cat /proc/meminfo


In [ ]:
#5. 디스크 사양
!df -h

#6. 파이썬 버전
!python --version

#7. GPU 사양
!nvidia-smi

!tensorflow.keras --version

In [ ]:
# 이후 학습에 사용할 디렉토리들이 있는 상위 디렉토리의 위치를 cat_dir변수에 저장
cat_dir='/content/drive/MyDrive/data/train/wav'
#cat_dir='/content/drive/MyDrive/data/test/wav'


In [ ]:
# train_cat list에 cat_dir안의 모든 dir명들을 넣어줌(이경우 class label이 곧 dirname)
train_cat= [f for f in os.listdir(cat_dir)]

In [ ]:
# train_cat의 내용을 확인
train_cat

['on']

In [ ]:
# MFCC 추출 및 데이터 전처리(sklearn.preprocessing.scale)
import sklearn.preprocessing 
from PIL import Image

def mfcc_extract(filename):
  y,sr  = librosa.load(filename,sr = 16000)
  mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=100,n_fft=int(0.025*sr),hop_length=int(0.01*sr))

  mfcc = sklearn.preprocessing.scale(mfcc, axis=1)
  pad2d = lambda a, i: a[:, 0:i] if a.shape[1] > i else np.hstack((a, np.zeros((a.shape[0], i-a.shape[1]))))
  padded_mfcc = pad2d(mfcc, 90)
  #librosa.display.specshow(padded_mfcc, sr=16000, x_axis='time')  #시각화하기

  #delta=librosa.feature.delta(mfcc)
  #delta2=librosa.feature.delta(mfcc,order=2)
  #con_mfcc=np.concatenate((mfcc,delta,delta2),axis=0)
  #print(y)
  #print(len(y))
  #print('Sampling rate (Hz): %d' %sr)
  #print('Audio length (seconds): %.2f' % (len(y) / sr)) #음악의 길이(초) = 음파의 길이/Sampling rate
  return padded_mfcc
  
# parse_audio_files는 parent_dir, sub_dirs를 변수로 받음
# parent_dir 아래의 sub_dirs안에서 wav파일들을 읽고 mfcc feature를 추출 
def parse_audio_files(parent_dir, sub_dirs):
  labels = []
  features = []
  for label,sub_dir in enumerate(tqdm(sub_dirs)):
    print(sub_dir)
    label = int( f"{sub_dir}".split(".")[0].strip() )
    for fn in glob.glob(os.path.join(parent_dir,sub_dir,"*.wav")):
      #print mfcc_extract(fn).shape
      features.append(mfcc_extract(fn))
      labels.append(label)

      fn_new = fn.replace('/wav/','/img/')
      fn_new = fn_new.replace('.wav','.png')
      
      plt.imsave( fn_new,  mfcc_extract(fn))
  
  return features,labels

In [ ]:
# File들로부터 feature를 추출
# 모든 음성파일들은 1초이며 sampling rate는 160kHz, bit는 16bit이다. (확인필요!!! - https://melon1024.github.io/ssc/)
# 따라서 mfcc feature들은 한 파일당 13*501의 형태를 가지게 된다.
features, labels = parse_audio_files(cat_dir,train_cat)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/1 [00:00<?, ?it/s]

on


/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:236: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  "Numerical issues were encountered "
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:255: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  "Numerical issues were encountered "
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:236: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  "Numerical issues were encountered "
/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py:255: UserWarning: Numerical issues were encountered when scaling the data and mig

In [ ]:
# features 확인 
# features는 python기본 타입 중 하나인 list이며 (13*501) ndarray를 원소로 가지고 있음

print(len(features), features[0].shape)
print(type(features), type(features[0]))

300 (100, 90)
<class 'list'> <class 'numpy.ndarray'>


In [ ]:
# Part 1 - Building the CNN 
# Importing the Keras libraries and packages

from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense


# Initialising the CNN
classifier = Sequential()

# Step 1 - Convolution
classifier.add(Conv2D(32, (3, 3), input_shape = (100, 90, 3), activation = 'relu'))

# Step 2 - Pooling
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Adding a second convolutional layer
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full connection
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Part 2 - Fitting the CNN to the images
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/data/train/img/',
                                                 target_size = (100, 90),
                                                 batch_size = 10,
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/data/test/img/',
                                            target_size = (100, 90),
                                            batch_size = 10,
                                            class_mode = 'binary')

# 데이터를 무작위로 섞는다.???? 넣으면 오류나나
#import random
#random.shuffle(training_set)
#random.shuffle(test_set)

classifier.fit(training_set,
                         steps_per_epoch = 30,
                         epochs = 50,
                         validation_data = test_set,
                         validation_steps = 3)

output = classifier.predict(test_set, steps=5)
print(test_set.class_indices)
print(output)

Found 1000 images belonging to 1 classes.
Found 300 images belonging to 1 classes.


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:63: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.


Epoch 1/50
30/30 [==============================] - 32s 72ms/step - loss: 0.0229 - accuracy: 1.0000 - val_loss: 1.6387e-27 - val_accuracy: 1.0000
Epoch 2/50
30/30 [==============================] - 2s 64ms/step - loss: 2.1007e-27 - accuracy: 1.0000 - val_loss: 2.2121e-29 - val_accuracy: 1.0000
Epoch 3/50
30/30 [==============================] - 2s 66ms/step - loss: 1.7568e-28 - accuracy: 1.0000 - val_loss: 4.1389e-30 - val_accuracy: 1.0000
Epoch 4/50
30/30 [==============================] - 2s 67ms/step - loss: 1.0716e-28 - accuracy: 1.0000 - val_loss: 7.7582e-30 - val_accuracy: 1.0000
Epoch 5/50
30/30 [==============================] - 2s 66ms/step - loss: 1.2004e-28 - accuracy: 1.0000 - val_loss: 7.1505e-30 - val_accuracy: 1.0000
Epoch 6/50
30/30 [==============================] - 2s 71ms/step - loss: 1.3017e-28 - accuracy: 1.0000 - val_loss: 6.9464e-30 - val_accuracy: 1.0000
Epoch 7/50
30/30 [==============================] - 2s 69ms/step - loss: 1.1402e-28 - accuracy: 1.0000 - val_

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:65: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.


{'on': 0}
[[1.15597896e-29]
 [2.54912326e-29]
 [6.85987809e-30]
 [2.01066757e-31]
 [4.38147934e-30]
 [3.39379913e-31]
 [8.43074423e-31]
 [3.49464408e-31]
 [4.97612939e-30]
 [7.41568040e-31]
 [3.10706314e-31]
 [3.45509011e-31]
 [2.45517399e-29]
 [2.34471796e-30]
 [3.49539463e-29]
 [3.73397868e-30]
 [1.65723921e-31]
 [1.04068020e-31]
 [8.10192686e-30]
 [1.13201280e-29]
 [9.72095096e-32]
 [3.31313388e-31]
 [6.11714922e-30]
 [4.87318543e-30]
 [1.18147451e-30]
 [1.52838189e-30]
 [2.99511176e-29]
 [5.90666644e-29]
 [1.81210056e-29]
 [4.36952879e-30]
 [1.48230806e-30]
 [4.91828973e-29]
 [3.02604760e-31]
 [5.44672737e-30]
 [2.81304309e-31]
 [5.68770888e-31]
 [4.68795846e-32]
 [2.48002201e-31]
 [7.67991889e-29]
 [6.11735583e-31]
 [1.56642488e-29]
 [2.34777150e-29]
 [1.46039628e-30]
 [4.33243066e-31]
 [1.27240528e-31]
 [1.25957576e-28]
 [4.61927902e-31]
 [1.53958855e-29]
 [1.23970185e-29]
 [2.18852833e-31]]
